In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from IPython.display import display

In [2]:
print('loading products...')
products = pd.read_csv('products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8})

print('loading prior orders...')
prior = pd.read_csv('order_products__prior.csv', dtype={
        'order_id': np.int32,
        'product_id': np.uint16,
        'add_to_cart_order': np.int16,
        'reordered': np.int8})

print('loading train orders...')
train = pd.read_csv('order_products__train.csv', dtype={
        'order_id': np.int32,
        'product_id': np.uint16,
        'add_to_cart_order': np.int16,
        'reordered': np.int8})

print('loading orders...')
order = pd.read_csv('orders.csv' , dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})


loading products...
loading prior orders...
loading train orders...
loading orders...


In [51]:
prior_orders = order[order.eval_set == 'prior']
train_orders = order[order.eval_set == 'train']
test_orders = order[order.eval_set == 'test']

In [52]:
users = pd.DataFrame()
users['orders'] = order.groupby(order.user_id)['order_id'].apply(list)
users['nb_order'] = order.groupby(order.user_id)['order_number'].size()
users['avg_days_since_prior'] = order.groupby(order.user_id)['days_since_prior_order'].mean().astype(np.float32)
users['avg_hour_of_day'] = order.groupby(order.user_id)['order_hour_of_day'].mean().astype(np.float32)
users['days_since_prior'] = train_orders['days_since_prior_order']


In [54]:
display(train_orders)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
49,2196797,5,train,5,0,11,6.0
74,525192,7,train,21,2,11,6.0
78,880375,8,train,4,1,14,10.0
82,1094988,9,train,4,6,10,30.0
88,1822501,10,train,6,0,19,30.0
115,1827621,13,train,13,0,21,8.0
129,2316178,14,train,14,2,19,11.0
200,2180313,17,train,41,3,10,30.0
